In [31]:
import pdbfixer
import mdtraj as md
from simtk.openmm.app import PDBFile

# 5UDC

In [ ]:
# Chain 0 : H
# Chain 1 : L
# Chain 2 : F 
# Chain 3 : B
# Chain 4 : C
# Chain 5 : A
# Chain 6 : E
# Chain 7 : G 
# Chain 8 : D 

In [87]:
# Load 5udc into MDTraj as a PDBTrajectoryFile
traj = md.formats.PDBTrajectoryFile('../data/5udc/5udc.pdb')

In [88]:
# Renumber residue numbers in each chain such that 
## inserted residues (e.g. 100, 100A, 100B, 100C) are incremented properly (i.e. 100, 101, 102, 103)
## the residues after inserted residues are adjusted properly (e.g. 101 --> 104)
## gaps for missing residues are maintained 

for chain in traj.topology.chains:
    previous_res_cur = 0
    previous_res_old = 0
    for res in chain.residues:
        res_old = res.resSeq
        if res.resSeq == previous_res_cur:
            res.resSeq += 1
        elif res.resSeq == previous_res_old:
            res.resSeq = previous_res_cur + 1
        elif res.resSeq < previous_res_cur:
            res.resSeq = previous_res_cur + (res_old - previous_res_old)
        previous_res_cur = res.resSeq
        previous_res_old = res_old
#         print(res, res_old, res.resSeq)

In [34]:
# Since MDTraj renames chains A, B, C, etc., rename them to match those in 5udc
traj.set_chain_names(['H', 'L', 'F', 'B', 'C', 'A', 'E', 'G', 'D'])

In [35]:
# Save to pdb file
out_file = md.formats.PDBTrajectoryFile("../data/5udc/5udc_mdtraj.pdb", mode='w')
out_file.write(traj.positions[0], traj.topology) # 0th frame 
out_file.close()

In [89]:
# Add SEQRES lines to 5udc_mdtraj.pdb manually

# Now, load the file into PDBFixer
fixer = pdbfixer.PDBFixer(filename='../data/5udc/5udc_mdtraj.pdb')

In [ ]:
# Remove unwanted chains
# fixer.removeChains(indices)

In [91]:
# Identify missing residues
fixer.findMissingResidues()
fixer.missingResidues

{(0, 0): ['GLN'],
 (0, 139): ['SER', 'LYS', 'SER', 'THR', 'SER', 'GLY'],
 (0, 217): ['GLU', 'PRO', 'LYS', 'SER'],
 (1, 211): ['GLY', 'GLU', 'CYS'],
 (2, 0): ['MET',
  'GLU',
  'LEU',
  'LEU',
  'ILE',
  'LEU',
  'LYS',
  'ALA',
  'ASN',
  'ALA',
  'ILE',
  'THR',
  'THR',
  'ILE',
  'LEU',
  'THR',
  'ALA',
  'VAL',
  'THR',
  'PHE',
  'CYS',
  'PHE',
  'ALA',
  'SER',
  'GLY',
  'GLN'],
 (2, 72): ['SER',
  'THR',
  'PRO',
  'ALA',
  'THR',
  'ASN',
  'ASN',
  'ARG',
  'ALA',
  'ARG',
  'ARG',
  'GLU',
  'LEU',
  'PRO',
  'ARG',
  'PHE',
  'MET',
  'ASN',
  'TYR',
  'THR',
  'LEU',
  'ASN',
  'ASN',
  'ALA',
  'LYS',
  'LYS',
  'THR',
  'ASN',
  'VAL',
  'THR',
  'LEU',
  'SER',
  'LYS',
  'LYS',
  'ARG',
  'LYS',
  'ARG',
  'ARG'],
 (2, 449): ['SER',
  'ALA',
  'ILE',
  'GLY',
  'GLY',
  'TYR',
  'ILE',
  'PRO',
  'GLU',
  'ALA',
  'PRO',
  'ARG',
  'ASP',
  'GLY',
  'GLN',
  'ALA',
  'TYR',
  'VAL',
  'ARG',
  'LYS',
  'ASP',
  'GLY',
  'GLU',
  'TRP',
  'VAL',
  'LEU',
  'LEU',
  'S

In [64]:
# Remove missing residues if they are part of terminal fragments 
chains = list(fixer.topology.chains())
keys = fixer.missingResidues.keys()
for key in list(keys): # Declare as list because makes a copy of the dict keys
    chain = chains[key[0]]
    if key[1] == 0 or key[1] == len(list(chain.residues())):
        if len(fixer.missingResidues[key]) > 10: # Do not add back terminal fragment if its longer than 10 residues
            print(key, chain)
            del fixer.missingResidues[key]


(8, 448) <Chain 8>
(5, 448) <Chain 5>
(8, 0) <Chain 8>
(2, 0) <Chain 2>
(2, 449) <Chain 2>
(5, 0) <Chain 5>


In [65]:
# Identify nonstandard residues
fixer.findNonstandardResidues()
fixer.nonstandardResidues
# fixer.replaceNonstandardResidues()

[]

In [66]:
# Remove heterogens

# The argument specifies whether to keep water molecules. 
# False removes all heterogens including water. 
# True keeps water molecules while removing all other heterogens.

fixer.removeHeterogens(False)


In [67]:
# Add missing heavy atoms

# findMissingAtoms() identifies all missing heavy atoms 
# and stores them into two fields called missingAtoms and missingTerminals. 
# Each of these is a dictionary whose keys are Residue objects and whose values are lists of atom names. 
# missingAtoms contains standard atoms that should be present in any residue of that type, 
# while missingTerminals contains missing terminal atoms that should be present at the start or end of a chain. 
# You are free to remove atoms from these dictionaries before continuing, if you want to prevent certain atoms 
# from being added.

fixer.findMissingAtoms()


In [68]:
fixer.missingAtoms

{}

In [69]:
fixer.missingTerminals

{<Residue 1753 (LEU) of chain 5>: ['OXT'],
 <Residue 1973 (SER) of chain 6>: ['OXT'],
 <Residue 2634 (LEU) of chain 8>: ['OXT'],
 <Residue 876 (LEU) of chain 2>: ['OXT']}

In [70]:
# addMissingAtoms() is the point at which all heavy atoms get added. 
# This includes the ones identified by findMissingAtoms() as well 
# as the missing residues identified by findMissingResidues(). 
# Also, if you used replaceNonstandardResidues() to modify any residues, 
# that will have removed any atoms that do not belong in the replacement residue, 
# but it will not have added ones that are missing from the original residue. 
# addMissingAtoms() is the point when those get added.

fixer.addMissingAtoms()

In [ ]:
# Add missing hydrogens
# fixer.addMissingHydrogens(7.0)

In [72]:
PDBFile.writeFile(fixer.topology, fixer.positions, open('../data/5udc/5udc_clean_nolongterms.pdb', 'w'))


# 4JHW

In [ ]:
# Chain 0 : H
## Missing 214-217

# Chain 1 : L
## Missing 212-214

# Chain 2 : F
## Missing 125-136 and 514-550

In [147]:
# Load 5udc into MDTraj as a PDBTrajectoryFile
traj = md.formats.PDBTrajectoryFile('../data/4jhw/4jhw_add_missing.pdb')

In [148]:
# Renumber residue numbers in each chain such that 
## inserted residues (e.g. 100, 100A, 100B, 100C) are incremented properly (i.e. 100, 101, 102, 103)
## the residues after inserted residues are adjusted properly (e.g. 101 --> 104)
## gaps for missing residues are maintained 

for chain in traj.topology.chains:
    previous_res_cur = 0
    previous_res_old = 0
    for res in chain.residues:
        res_old = res.resSeq
        if res.resSeq == previous_res_cur:
            res.resSeq += 1
        elif res.resSeq == previous_res_old:
            res.resSeq = previous_res_cur + 1
        elif res.resSeq < previous_res_cur:
            res.resSeq = previous_res_cur + (res_old - previous_res_old)
        previous_res_cur = res.resSeq
        previous_res_old = res_old
#         print(res, res_old, res.resSeq)

In [150]:
# Since MDTraj renames chains A, B, C, etc., rename them to match those in 5udc
traj.set_chain_names(['H', 'L', 'F'])

In [165]:
# Save to pdb file
out_file = md.formats.PDBTrajectoryFile("../data/4jhw/4jhw_mdtraj_add_missing.pdb", mode='w')
out_file.write(traj.positions[0], traj.topology) # 0th frame 
out_file.close()

In [180]:
# Add SEQRES lines to 5udc_mdtraj.pdb manually

# Now, load the file into PDBFixer
fixer = pdbfixer.PDBFixer(filename='../data/4jhw/4jhw_mdtraj_add_missing.pdb')

In [182]:
# Remove unwanted chains
# fixer.removeChains([2])

In [188]:
# Identify missing residues
fixer.findMissingResidues()
fixer.missingResidues

{(0, 226): ['LYS', 'SER', 'CYS', 'ASP'],
 (1, 211): ['GLY', 'GLU', 'CYS'],
 (2, 72): ['GLN',
  'SER',
  'THR',
  'PRO',
  'ALA',
  'THR',
  'ASN',
  'ASN',
  'ARG',
  'ALA',
  'ARG',
  'ARG',
  'GLU',
  'LEU',
  'PRO',
  'ARG',
  'PHE',
  'MET',
  'ASN',
  'TYR',
  'THR',
  'LEU',
  'ASN',
  'ASN',
  'ALA',
  'LYS',
  'LYS',
  'THR',
  'ASN',
  'VAL',
  'THR',
  'LEU',
  'SER',
  'LYS',
  'LYS',
  'ARG',
  'LYS',
  'ARG',
  'ARG'],
 (2, 449): ['SER',
  'ALA',
  'ILE',
  'GLY',
  'GLY',
  'TYR',
  'ILE',
  'PRO',
  'GLU',
  'ALA',
  'PRO',
  'ARG',
  'ASP',
  'GLY',
  'GLN',
  'ALA',
  'TYR',
  'VAL',
  'ARG',
  'LYS',
  'ASP',
  'GLY',
  'GLU',
  'TRP',
  'VAL',
  'LEU',
  'LEU',
  'SER',
  'THR',
  'PHE',
  'LEU',
  'GLY',
  'GLY',
  'LEU',
  'VAL',
  'PRO',
  'ARG']}

In [189]:
# Remove missing residues if they are part of terminal fragments 
chains = list(fixer.topology.chains())
keys = fixer.missingResidues.keys()
for key in list(keys): # Declare as list because makes a copy of the dict keys
    chain = chains[key[0]]
    if key[1] == 0 or key[1] == len(list(chain.residues())):
        if len(fixer.missingResidues[key]) > 10: # Do not add back terminal fragment if its longer than 10 residues
            print(key, chain)
            del fixer.missingResidues[key]


(2, 449) <Chain 2>


In [190]:
# Identify nonstandard residues
fixer.findNonstandardResidues()
fixer.nonstandardResidues
# fixer.replaceNonstandardResidues()

[]

In [191]:
# Remove heterogens

# The argument specifies whether to keep water molecules. 
# False removes all heterogens including water. 
# True keeps water molecules while removing all other heterogens.

fixer.removeHeterogens(False)


In [192]:
# Add missing heavy atoms

# findMissingAtoms() identifies all missing heavy atoms 
# and stores them into two fields called missingAtoms and missingTerminals. 
# Each of these is a dictionary whose keys are Residue objects and whose values are lists of atom names. 
# missingAtoms contains standard atoms that should be present in any residue of that type, 
# while missingTerminals contains missing terminal atoms that should be present at the start or end of a chain. 
# You are free to remove atoms from these dictionaries before continuing, if you want to prevent certain atoms 
# from being added.

fixer.findMissingAtoms()


In [193]:
fixer.missingAtoms

{}

In [194]:
fixer.missingTerminals

{<Residue 885 (LEU) of chain 2>: ['OXT']}

In [195]:
# addMissingAtoms() is the point at which all heavy atoms get added. 
# This includes the ones identified by findMissingAtoms() as well 
# as the missing residues identified by findMissingResidues(). 
# Also, if you used replaceNonstandardResidues() to modify any residues, 
# that will have removed any atoms that do not belong in the replacement residue, 
# but it will not have added ones that are missing from the original residue. 
# addMissingAtoms() is the point when those get added.

fixer.addMissingAtoms()

In [ ]:
# Add missing hydrogens
# fixer.addMissingHydrogens(7.0)

In [196]:
PDBFile.writeFile(fixer.topology, fixer.positions, open('../data/4jhw/4jhw_clean.pdb', 'w'))
